In [95]:
import urllib
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from os.path import join, exists
from os import  remove, makedirs
from time import gmtime, strftime

pd_symbol = 'dohome'

DIR_CURRENT = os.path.dirname(os.path.abspath("__file__"))
DIR_SEC_CSV = "sec_set_price"

def loadPriceData(symbol_list, dates,  base_dir=DIR_SEC_CSV):
    return loadManySymbols(symbol_list, dates, 'Close', base_dir)

def loadManySymbols(symbols, dates, column_name, base_dir):
    df = pd.DataFrame(index=dates)	# empty data frame that has indexs as dates
        
    base_dir = join(DIR_CURRENT,base_dir)
    for symbol in symbols:
        csv_file = os.path.join(base_dir, symbol + '.csv')
        df_temp = pd.read_csv(csv_file, index_col='Date',
            parse_dates=False, usecols=['Date', column_name], 
            na_values=['nan'])
        df_temp = df_temp.rename(columns={column_name: symbol})
        df = df.join(df_temp) # left join by default
        df = df.dropna()
#         print('df test', df)
    return df

from stock_pandas import StockDataFrame

startDate = '2017-03-01'
endDate = strftime("%Y-%m-%d", gmtime())
dates = pd.date_range(startDate, endDate).strftime('%d/%m/%Y')
df_symbol = loadPriceData([pd_symbol], dates)
# print('df_symbol', df_symbol.tail(20))
stock = StockDataFrame(df_symbol)

stock.alias('close', pd_symbol)
d = stock[['macd','macd:26','macd.histogram:16', 'close']].rename(columns={'macd.histogram:16,26,9':'sig',
                                                                                             }).tail(50)
# d = stock.alias('Open', 'macd.histogram:16,26,9')

In [93]:
# d['x'] = d['macd.histogram:16,26,9'].shift(-1)
d['sig2'] = d['sig'].shift(-1)
d['is_interest'] = d.apply(
    (lambda row: (row['sig2'] > row['sig'])),
    axis=1
)

In [94]:
print(d)

            macd:12,26  macd:26,26       sig  global      sig2  is_interest
14/09/2020    0.182627         0.0 -0.250791    18.8 -0.283839        False
15/09/2020    0.098209         0.0 -0.283839    18.8 -0.255318         True
16/09/2020    0.078627         0.0 -0.255318    19.4 -0.228889         True
17/09/2020    0.062390         0.0 -0.228889    19.4 -0.190719         True
18/09/2020    0.064858         0.0 -0.190719    19.6 -0.187800         True
21/09/2020    0.034243         0.0 -0.187800    19.2 -0.145308         True
22/09/2020    0.049638         0.0 -0.145308    19.7 -0.127106         True
23/09/2020    0.045221         0.0 -0.127106    19.5 -0.111474         True
24/09/2020    0.041246         0.0 -0.111474    19.5 -0.111895        False
25/09/2020    0.021743         0.0 -0.111895    19.3 -0.060093         True
28/09/2020    0.061938         0.0 -0.060093    20.0 -0.017976         True
29/09/2020    0.100684         0.0 -0.017976    20.1  0.016009         True
30/09/2020  